### Building a RAG System with LangChain and FAISS 
Introduction to RAG (Retrieval-Augmented Generation)
RAG combines the power of retrieval systems with generative AI models. Instead of relying solely on the model's training data, RAG:

1. Retrieves relevant documents from a knowledge base
2. Uses these documents as context for the LLM
3. Generates responses based on both the retrieved context and the model's knowledge

### FAISS 
https://github.com/facebookresearch/faiss

FAISS is a library for efficient similarity search and clustering of dense vectors.

Key advantages:
1. Extremely fast similarity search
2. Memory efficient
3. Supports GPU acceleration
4. Can handle millions of vectors

How it works:
- Indexes vectors for fast nearest neighbor search
- Returns most similar vectors based on distance metrics


In [2]:
## load libraries
import os
from dotenv import load_dotenv
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# LangChain core imports
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import (
    RunnablePassthrough, 
 
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

# LangChain specific imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Load environment variables
load_dotenv()

True

### Data Ingestion And Processing


In [3]:
sample_documents = [
    Document(
        page_content="""
        Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.
        """,
        metadata={"source": "AI Introduction", "page": 1, "topic": "AI"}
    ),
    Document(
        page_content="""
        Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.
        """,
        metadata={"source": "ML Basics", "page": 1, "topic": "ML"}
    ),
    Document(
        page_content="""
        Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning has revolutionized computer vision, NLP, and speech recognition.
        """,
        metadata={"source": "Deep Learning", "page": 1, "topic": "DL"}
    ),
    Document(
        page_content="""
        Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
        It combines computational linguistics with machine learning and deep learning models.
        Applications include chatbots, translation, sentiment analysis, and text summarization.
        """,
        metadata={"source": "NLP Overview", "page": 1, "topic": "NLP"}
    )
]

print(sample_documents)

[Document(metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}, page_content='\n        Artificial Intelligence (AI) is the simulation of human intelligence in machines.\n        These systems are designed to think like humans and mimic their actions.\n        AI can be categorized into narrow AI and general AI.\n        '), Document(metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='\n        Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.\n        '), Document(metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='\n        Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolu

In [4]:
## text splitting
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=[" "]
)

## split the documents into chunks
chunks = text_splitter.split_documents(sample_documents)
print(chunks[0])
print(chunks[1])


page_content='Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.' metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}
page_content='Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.' metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}


In [5]:

print(f"Created {len(chunks)} chunks from {len(sample_documents)} documents")
print("\nExample chunk:")
print(f"Content: {chunks[0].page_content}")
print(f"Metadata: {chunks[0].metadata}")

Created 4 chunks from 4 documents

Example chunk:
Content: Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.
Metadata: {'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}


In [6]:
### load the embedding models
import os
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [7]:
# Initialize OpenAI embeddings with the latest model

embeddings=OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=1536
)

## Example: create a embedding for a single text
sample_text="What is machine learning"
sample_embedding=embeddings.embed_query(sample_text)
sample_embedding

[-0.0059221177361905575,
 -0.005889697000384331,
 0.0005751235294155777,
 -0.033544257283210754,
 0.021192103624343872,
 0.02213229238986969,
 -0.0007868689717724919,
 0.00929923728108406,
 -0.022564563900232315,
 0.037823744118213654,
 0.015183531679213047,
 -0.03676467761397362,
 -0.03397652879357338,
 0.0016048074467107654,
 0.026735983788967133,
 0.016015654429793358,
 0.0008530605118721724,
 -0.030864175409078598,
 0.023385880514979362,
 0.004903578199446201,
 3.242035018047318e-05,
 0.032550033181905746,
 0.042276136577129364,
 -0.029307996854186058,
 0.015961619094014168,
 -0.02092193439602852,
 0.0266279149800539,
 0.01239538099616766,
 0.00744047062471509,
 -0.006019378546625376,
 -0.009423515759408474,
 -0.029027020558714867,
 -0.03127483278512955,
 0.02755729854106903,
 0.04173579812049866,
 0.00043159592314623296,
 -0.0080564571544528,
 -0.010633875615894794,
 -0.055503640323877335,
 0.0025855230633169413,
 -0.05632495880126953,
 -0.016264209523797035,
 0.03028060868382454,

In [8]:
texts=["AI","MAchine learning","Deep Learning","Neural Network"]
batch_embeddings=embeddings.embed_documents(texts)
print(batch_embeddings[0])

[-0.008148186840116978, -0.024675853550434113, 0.002891185460612178, 0.025217745453119278, 0.0065126013942062855, -0.02828405424952507, -0.005048835650086403, 0.020988354459404945, -0.036875005811452866, 0.012840167619287968, -0.0030250060372054577, -0.020129257813096046, 0.00026144576258957386, -0.03275134786963463, 0.0064762551337480545, -0.0252706129103899, -0.031006723642349243, -0.054347675293684006, 0.032777782529592514, -0.018358200788497925, 0.016640011221170425, 0.048320792615413666, -0.024913758039474487, 0.01435349602252245, 0.029367834329605103, 0.004070789087563753, 0.009291443973779678, 0.013355623930692673, 0.002506244694814086, -0.022574376314878464, 0.03216980770230293, -0.028046149760484695, 0.005359431728720665, -0.03819669038057327, -0.01674574613571167, 0.01435349602252245, -0.03861963003873825, -0.010375224985182285, -0.01052721869200468, -0.019164428114891052, 0.03206407278776169, 0.014604616910219193, -0.02151702716946602, 0.016111336648464203, -0.01182247046381

In [9]:
print(batch_embeddings[1])

[-0.018223760649561882, 0.010756313800811768, 0.01748744770884514, -0.035269420593976974, 0.03742927312850952, 0.026090044528245926, 0.01585528440773487, 0.008381702937185764, -0.010903576388955116, 0.03487671911716461, -0.006645229645073414, -0.0686735138297081, -0.03234870731830597, 0.008958481252193451, 0.036521151661872864, 0.018726907670497894, 0.016321616247296333, -0.015413496643304825, 0.020604507997632027, 0.013008206151425838, -0.002647660905495286, 0.037821974605321884, 0.03406677395105362, -0.030017049983143806, 0.02061677910387516, 0.007939914241433144, 0.036103907972574234, 0.037674710154533386, 0.010535419918596745, -0.03482763096690178, -0.0109342560172081, -0.024224715307354927, -0.013989957980811596, -0.013867238536477089, 0.01194668747484684, -0.0018269780557602644, -0.02758721448481083, 0.027832651510834694, -0.045062389224767685, -0.005184874404221773, -0.019720930606126785, -0.042779818177223206, 0.034385841339826584, 0.0682317242026329, -0.02057996392250061, -0.0

In [10]:
### Compare Embedding using cosine similarity

def compare_embeddings(text1:str,text2:str):
    """Compare semantic simialrity of 2 texts usign embeddings"""

    emb1=np.array(embeddings.embed_query(text1))
    emb2=np.array(embeddings.embed_query(text2))

    ## Calculate the simialrity score

    similarity=np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
    return similarity

In [11]:
# Test semantic similarity
print("\nSemantic Similarity Examples:")
print(f"'AI' vs 'Artificial Intelligence': {compare_embeddings('AI', 'Artificial Intelligence'):.3f}")


Semantic Similarity Examples:
'AI' vs 'Artificial Intelligence': 0.563


In [12]:
print(f"'AI' vs 'Pizza': {compare_embeddings('AI', 'Pizza'):.3f}")

'AI' vs 'Pizza': 0.254


In [13]:
print(f"'Machine Learning' vs 'ML': {compare_embeddings('Machine Learning', 'ML'):.3f}")

'Machine Learning' vs 'ML': 0.461


### Create FAISS Vector Store

In [14]:
vectorstore=FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)
print(f"Vector store created with {vectorstore.index.ntotal} vectors")

Vector store created with 4 vectors


In [15]:
vectorstore

In [16]:
## Save vector tore for later use
vectorstore.save_local("faiss_index")
print("Vector store saved to 'faiss_index' directory")

Vector store saved to 'faiss_index' directory


In [ ]:
## load vector store
loaded_vectorstore=FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True  # This is needed for loading the index safely in some cases
)

print(f"Loaded vector store contains {loaded_vectorstore.index.ntotal} vectors")

Loaded vector store contains 4 vectors


In [18]:
## Similarity Search 
query="What is deep learning"

results=vectorstore.similarity_search(query,k=3)
print(results)

[Document(id='b1f8720b-f9cb-46b0-a6d1-5fa15bf5dba3', metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolutionized computer vision, NLP, and speech recognition.'), Document(id='04f8a6f9-cb67-4655-ae40-3180ccf2155a', metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.'), Document(id='9bcd7e7f-e1fc-49ca-b9fd-76d34fedecbf', metadata={'source': 'NLP Overview', 'page': 1, 'topic': 'NLP'}, page_content='Natural Language Processing (NLP) is a branch of AI that helps computers understand human lang

In [19]:
print(f"Query: {query}\n")
print("Top 3 similar chunks:")
for i, doc in enumerate(results):
    print(f"\n{i+1}. Source: {doc.metadata['source']}")
    print(f"   Content: {doc.page_content[:200]}...")

Query: What is deep learning

Top 3 similar chunks:

1. Source: Deep Learning
   Content: Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning ...

2. Source: ML Basics
   Content: Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised...

3. Source: NLP Overview
   Content: Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
        It combines computational linguistics with machine learning and deep learning models.
      ...


In [20]:
### Similarity Search with score
results_with_scores=vectorstore.similarity_search_with_score(query,k=3)

print("\n\nSimilarity search with scores:")
for doc, score in results_with_scores:
    print(f"\nScore: {score:.3f}")
    print(f"Source: {doc.metadata['source']}")
    print(f"Content preview: {doc.page_content[:100]}...")



Similarity search with scores:

Score: 0.556
Source: Deep Learning
Content preview: Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses m...

Score: 1.208
Source: ML Basics
Content preview: Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being...

Score: 1.274
Source: NLP Overview
Content preview: Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
...


In [21]:
chunks

[Document(metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}, page_content='Artificial Intelligence (AI) is the simulation of human intelligence in machines.\n        These systems are designed to think like humans and mimic their actions.\n        AI can be categorized into narrow AI and general AI.'),
 Document(metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.'),
 Document(metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolutionized computer vision, NLP, and speech recogn

In [22]:
### Search with metadata filtering
filter_dict={"topic":"ML"}
filtered_results=vectorstore.similarity_search(
    query,
    k=3,
    filter=filter_dict
)
print(filtered_results)

[Document(id='04f8a6f9-cb67-4655-ae40-3180ccf2155a', metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.')]


In [23]:
len(filtered_results)

1

### Build RAG Chain With LCEL 

In [24]:
## LLM GROQ LLM
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=init_chat_model(model="groq:openai/gpt-oss-20b")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x116cd5100>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x117490b60>, model_name='openai/gpt-oss-20b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [25]:
llm.invoke("Hi")

AIMessage(content='Hello! 👋 How can I help you today?', additional_kwargs={'reasoning_content': 'We need to respond to user greeting. Should be friendly.'}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 72, 'total_tokens': 105, 'completion_time': 0.027626827, 'prompt_time': 0.005109743, 'queue_time': 0.123699401, 'total_time': 0.03273657}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_fd68d84dd3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--ae877ca6-a6c0-4078-bbbf-78975715e7c2-0', usage_metadata={'input_tokens': 72, 'output_tokens': 33, 'total_tokens': 105})

In [26]:
# 1. Simple RAG Chain with LCEL
simple_prompt = ChatPromptTemplate.from_template("""Answer the question based only on the following context:
Context: {context}

Question: {question}

Answer:""")

In [27]:
## Basic retriever
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)

In [28]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x116283950>, search_kwargs={'k': 3})

In [29]:
from typing import List
# Format documents for the prompt
def format_docs(docs: List[Document]) -> str:
    """Format documents for insertion into prompt"""
    formatted = []
    for i, doc in enumerate(docs):
        source = doc.metadata.get('source', 'Unknown')
        formatted.append(f"Document {i+1} (Source: {source}):\n{doc.page_content}")
    return "\n\n".join(formatted)

In [30]:
simple_rag_chain=(
    {"context":retriever | format_docs,"question":RunnablePassthrough() }
    | simple_prompt
    | llm
    |StrOutputParser()

)

In [31]:
simple_rag_chain

{
  context: VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x116283950>, search_kwargs={'k': 3})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\nContext: {context}\n\nQuestion: {question}\n\nAnswer:'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x116cd5100>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x117490b60>, model_name='openai/gpt-oss-20b', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()

In [32]:
### Conversational RAg Chain

conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the provided context to answer questions."),
    ("placeholder", "{chat_history}"),
    ("human", "Context: {context}\n\nQuestion: {input}"),
])

In [33]:
def create_conversational_rag():
    """Create a conversational RAG chain with memory"""
    return (
        RunnablePassthrough.assign(
            context=lambda x: format_docs(retriever.invoke(x["input"]))
        )
        | conversational_prompt
        | llm
        | StrOutputParser()
    )

conversational_rag = create_conversational_rag()

In [34]:
conversational_rag

RunnableAssign(mapper={
  context: RunnableLambda(lambda x: format_docs(retriever.invoke(x['input'])))
})
| ChatPromptTemplate(input_variables=['context', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag=

In [35]:
### streaming RAG chain
streaming_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | simple_prompt
    | llm
)

print("Modern RAG chains created successfully!")
print("Available chains:")
print("- simple_rag_chain: Basic Q&A")
print("- conversational_rag: Maintains conversation history")
print("- streaming_rag_chain: Supports token streaming")

Modern RAG chains created successfully!
Available chains:
- simple_rag_chain: Basic Q&A
- conversational_rag: Maintains conversation history
- streaming_rag_chain: Supports token streaming


In [36]:
# Test function for different chain types
def test_rag_chains(question: str):
    """Test all RAG chain variants"""
    print(f"Question: {question}")
    print("=" * 80)
    
    # 1. Simple RAG
    print("\n1. Simple RAG Chain:")
    answer = simple_rag_chain.invoke(question)
    print(f"Answer: {answer}")

    print("\n2. Streaming RAG:")
    print("Answer: ", end="", flush=True)
    for chunk in streaming_rag_chain.stream(question):
        print(chunk.content, end="", flush=True)
    print()

In [37]:
test_rag_chains("What is the difference between AI and machine learning")

Question: What is the difference between AI and machine learning

1. Simple RAG Chain:
Answer: Artificial Intelligence (AI) is the broader field that focuses on building systems that simulate human intelligence—machines that can think, reason, and mimic human actions. Machine Learning (ML) is a **subset** of AI; it is the specific approach in which machines learn patterns from data instead of being explicitly programmed. Thus, AI encompasses all intelligent behavior, while ML is one technique within AI that relies on data‑driven learning.

2. Streaming RAG:
Answer: Artificial Intelligence (AI) is the broader field that focuses on building systems that simulate human intelligence—machines that can think, reason, and mimic human actions. Machine Learning (ML) is a **subset** of AI; it is the specific approach in which machines learn patterns from data instead of being explicitly programmed. Thus, AI encompasses all intelligent behavior, while ML is one technique within AI that relies on 

In [40]:
import time

# Extended test function for timing comparison
def test_rag_chains_with_timing(question: str):
    """Test RAG chains with timing for invoke vs stream"""
    print(f"\nQuestion: {question}")
    print("=" * 80)
    
    # 1. Simple RAG (invoke)
    print("\n1️⃣ Simple RAG Chain (invoke):")
    start = time.time()
    answer = simple_rag_chain.invoke(question)
    end = time.time()
    print(f"Answer: {answer}")
    print(f"⏱️ Total time (invoke): {end - start:.2f} seconds")

    # 2. Streaming RAG
    print("\n2️⃣ Streaming RAG Chain (stream):")
    print("Answer: ", end="", flush=True)
    start = time.time()
    first_token_time = None

    for chunk in streaming_rag_chain.stream(question):
        if first_token_time is None:
            first_token_time = time.time()
            print(f"\n⚡ First token after {first_token_time - start:.2f} sec\n")
        print(chunk.content, end="", flush=True)

    end = time.time()
    print("\n")
    print(f"⏱️ Total time (stream): {end - start:.2f} seconds")
    if first_token_time:
        print(f"⚡ First token latency: {first_token_time - start:.2f} seconds")


In [41]:
test_rag_chains_with_timing("What is the difference between AI and machine learning?")



Question: What is the difference between AI and machine learning?

1️⃣ Simple RAG Chain (invoke):
Answer: Artificial Intelligence (AI) is the broader field that simulates human intelligence in machines—enabling systems to think and act like humans.  
Machine learning (ML) is a subset of AI; it is a specific set of algorithms that learn patterns from data instead of being explicitly programmed. Thus, ML is one way to achieve AI, but AI encompasses more than just machine‑learning techniques.
⏱️ Total time (invoke): 1.29 seconds

2️⃣ Streaming RAG Chain (stream):
Answer: 
⚡ First token after 0.93 sec

Artificial Intelligence (AI) is the broader field that simulates human intelligence in machines—enabling systems to think and act like humans.  
Machine learning (ML) is a subset of AI; it is a specific set of algorithms that learn patterns from data instead of being explicitly programmed. Thus, ML is one way to achieve AI, but AI encompasses more than just machine‑learning techniques.

⏱️ 

In [42]:
# Test with multiple questions
test_questions = [
    "What is the difference between AI and Machine Learning?",
    "Explain deep learning in simple terms",
    "How does NLP work?"
]

for question in test_questions:
    print("\n" + "=" * 80 + "\n")
    test_rag_chains(question)



Question: What is the difference between AI and Machine Learning?

1. Simple RAG Chain:
Answer: Artificial Intelligence (AI) is the overarching field that seeks to create machines capable of simulating human intelligence—thinking, reasoning, and acting like humans.  
Machine Learning (ML) is a **subset of AI**: it is a specific set of algorithms that enable a system to learn patterns from data instead of being explicitly programmed.  

In short: **AI is the broad goal of replicating human-like intelligence; ML is one of the techniques—learning from data—that accomplishes part of that goal.**

2. Streaming RAG:
Answer: Artificial Intelligence (AI) is the overarching field that seeks to create machines capable of simulating human intelligence—thinking, reasoning, and acting like humans.  
Machine Learning (ML) is a **subset of AI**: it is a specific set of algorithms that enable a system to learn patterns from data instead of being explicitly programmed.  

In short: **AI is the broad 

In [43]:
## Conversational example
print("\n3. Conversational RAG Example:")
chat_history = []

# First question
q1 = "What is machine learning?"
a1 = conversational_rag.invoke({
    "input": q1,
    "chat_history": chat_history
})

print(f"Q1: {q1}")
print(f"A1: {a1}")


3. Conversational RAG Example:
Q1: What is machine learning?
A1: **Machine learning** is a subset of artificial intelligence that allows computer systems to learn from data rather than being explicitly programmed. By analyzing input data, machine‑learning algorithms discover patterns and relationships, enabling the system to make predictions or decisions automatically. Common approaches include supervised learning, unsupervised learning, and reinforcement learning.


In [44]:
# Update history
chat_history.extend([
    HumanMessage(content=q1),
    AIMessage(content=a1)
])

In [45]:
# Follow-up question
q2 = "How is it different from traditional programming?"
a2 = conversational_rag.invoke({
    "input": q2,
    "chat_history": chat_history
})
print(f"\nQ2: {q2}")
print(f"A2: {a2}")


Q2: How is it different from traditional programming?
A2: ### Key Differences Between Machine Learning (ML) and Traditional Programming

| Aspect | Traditional Programming | Machine Learning |
|-------|-----------------------|----------------|
| **How the System Behaves** | **Explicitly coded rules** – the programmer writes every decision, loop, and conditional statement that dictates how the program should react to each input. | **Implicitly learned behavior** – the programmer supplies a *model* and *data*. The model discovers patterns and rules from that data, then applies them to new inputs. |
| **Source of Knowledge** | Programmer’s knowledge and intent. | Data-driven knowledge extracted from examples. |
| **Adaptability** | Rigid; changes require editing code. | Adaptive; can improve automatically as more data become available (unless the model is fixed). |
| **Development Cycle** | **Design‑code‑test**: write code, debug, test. | **Train‑evaluate‑tune**: gather data, train model